In [ ]:
from pepper_2d_simulator import *
from timeit import default_timer as timer
import pose2d
from copy import deepcopy

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook

In [ ]:
map_ = Map2D(".", "office_full")
start = timer()
map_8ds = map_.as_coarse_map2d().as_coarse_map2d().as_coarse_map2d()
print("8x Downsampling: {}s".format(timer()- start))

In [ ]:
def gridshow(*args, **kwargs):
    if not 'origin' in kwargs:
        kwargs['origin'] = 'lower'
    return plt.imshow(*(arg.T if i == 0 else arg for i, arg in enumerate(args)), **kwargs)

In [ ]:
kNMessages = 10
kPepperWidth = 0.48
kPepperComfortZoneRadius = 0.5
kDijkstraTSDFPenalty = 10.
kLocalAreaLimits = np.array([[-3., 5.],
                             [-3., 3.]])

## Debug Local TSDF

In [ ]:
from pyniel.ros_tools.quickros import QuickSubscriber
from sensor_msgs.msg import LaserScan
q = QuickSubscriber("sick_laser_front/cropped_scan", LaserScan, kNMessages,
                    tf_parent_frame="odom")

In [ ]:
scan = q.messages[0]
single_scan_map = Map2D()
start = timer()
scan.ranges = [1000. if r == 0 else r for r in scan.ranges] # find far away points
single_scan_map.from_scan(scan, map_.resolution(), kLocalAreaLimits)
print("Reverse raytrace: {} [s]".format(timer() - start))

In [ ]:
plt.figure()
gridshow(single_scan_map.occupancy(), cmap=plt.cm.Greys)

In [ ]:
import tf
poses = np.array([np.array([
      pose[0][0],
      pose[0][1], 
      tf.transformations.euler_from_quaternion(pose[1])[2]
                  ]) for pose in q.tfs])

In [ ]:
from map2d import LocalMap2D
local_map = LocalMap2D(kLocalAreaLimits, map_.resolution(),
                        sensor_model={"p_hit": 0.75, "p_miss": 0.25},
                        max_observations=kNMessages)
start = timer()
for scan, pose in zip(q.messages, poses):
  local_map.add_observation(scan, pose, pose)
tic = timer()
local_map2 = local_map.copy()
toc = timer()
local = local_map2.generate()
print("LocalMap2D Generation: {} s".format(timer() - start))
print("  Copy: {} s".format(toc - tic))

In [ ]:
# for visualization purposes
# poses in local map frame
current_pose = poses[-1]
r = np.array([ # rotation for oTa inverse, o = odom frame, a = current_scan frame
    [np.cos(current_pose[2]), np.sin(current_pose[2])],
    [-np.sin(current_pose[2]), np.cos(current_pose[2])],
    ])
rel_poses = []
for pose in poses:
    rel_pose = pose - current_pose
    rel_pose[:2] = np.sum(r * rel_pose[:2], axis = -1) # rel_pose x y in current_scan frame
    rel_poses.append(rel_pose)
rel_poses = np.array(rel_poses)
# rel_poses in local map grid
x_ = (rel_poses[:,0] - local_map2.origin[0]) / local_map2.resolution()
y_ = (rel_poses[:,1] - local_map2.origin[1])/ local_map2.resolution()
u_ = np.cos(rel_poses[:,2])/ local_map2.resolution()
v_ = np.sin(rel_poses[:,2])/ local_map2.resolution()

In [ ]:
plt.figure()
gridshow(local.occupancy())
plt.quiver(x_, y_, u_, v_, color='red')

#plt.figure()
#gridshow(err[2])

In [ ]:
start = timer()
submaps = [Map2D() for i in range(len(q.messages))]
for scan, submap in zip(q.messages, submaps):
    scan.ranges = [1000. if r == 0 else r for r in scan.ranges] # find far away points
    submap.from_scan(scan, map_.resolution(), kLocalAreaLimits)
print("Reverse raytrace on {} messages: {} [s]".format(kNMessages, timer() - start))

In [ ]:
# legacy map combining for performance reference
tic = timer()
incr_map = np.array([submap.occupancy() for submap in submaps])
incr_map[incr_map == 0] = 0.25
incr_map[incr_map == 1] = 0.75
print(timer() - tic)
final_map = np.prod(incr_map + 0.5, axis=0 )
def new_odds(latest, old):
    combined_odds = old / ( 1 - old ) * latest / ( 1 - latest )
    return np.clip(combined_odds / ( 1 + combined_odds ) ,1e-10,1-1e-10)
start = timer()
final_map = reduce(new_odds, incr_map)
print("Combining {} maps into one: {} s".format(kNMessages, timer() - start))
plt.figure()
gridshow(final_map)

In [ ]:
start = timer()
x_entropy_error, latest_hits = local_map2.cross_entropy_error(local_map2.observations[local_map2.ci_(0)],
                               np.array([0,0,0]),
                               local.occupancy())
print("Cross-entropy error: {}".format(timer() - start))

In [ ]:
plt.figure()
plt.plot(x_entropy_error)
plt.figure()
gridshow(local.occupancy())
plt.scatter(latest_hits[:,0], latest_hits[:,1], c=x_entropy_error, s=1, cmap=plt.cm.rainbow)

## Debug Global Planner

In [ ]:
# memory intensive!
start = timer()
sdf = map_8ds.as_sdf()
print("ESDF on 8x downsampled map: {}s".format(timer()- start))

start = timer()
tsdf = map_.as_tsdf(0.5)
print("TSDF on full-res map: {}s".format(timer()- start))

In [ ]:
plt.figure()
gridshow(sdf)
plt.tight_layout()

In [ ]:
plt.figure()
gridshow(tsdf)
plt.tight_layout()

In [ ]:
plt.figure()
traversable_sdf = sdf * (sdf > (kPepperWidth * 0.5))
gridshow(traversable_sdf)
plt.tight_layout()

In [ ]:
sdf_dx, sdf_dy = np.gradient(traversable_sdf)
plt.figure()
gridshow(sdf_dx)
plt.tight_layout()
plt.figure()
gridshow(sdf_dy)
plt.tight_layout()

In [ ]:
start = timer()
dijkstra = map_8ds.dijkstra(np.array([288, 100]), mask=sdf < 0,
                           extra_costs = kDijkstraTSDFPenalty*(kPepperComfortZoneRadius - sdf) * (sdf < kPepperComfortZoneRadius))
max_dijkstra = dijkstra[dijkstra < dijkstra.max()].max()
print(timer()- start)

In [ ]:
#start = timer()
#fdijkstra = map_.dijkstra(np.array([2800, 1300]), mask=tsdf < 0,
#                           extra_costs = kDijkstraTSDFPenalty*(kPepperComfortZoneRadius - tsdf) * (tsdf < kPepperComfortZoneRadius))
#print(timer()- start)

In [ ]:
plt.figure()
gridshow(dijkstra, vmax=dijkstra[dijkstra < dijkstra.max()].max())
plt.tight_layout()

In [ ]:
#plt.figure()
#gridshow(fdijkstra, vmax=fdijkstra[fdijkstra < fdijkstra.max()].max())
#plt.tight_layout()

In [ ]:
kSDFDiscount = 0.9 # [1/sqrt(2) to 1[

#sdf_cost = - sdf * kSDFDiscount + (kPepperComfortZoneRadius - sdf) * (sdf < kPepperComfortZoneRadius) + 100000 * (sdf <  (kPepperWidth  * 0.5))
costmap = dijkstra

plt.figure()
gridshow(costmap, vmax=max_dijkstra)
plt.tight_layout()

In [ ]:
grad_xx, grad_yy = np.gradient(costmap)
keepmask = np.where(traversable_sdf > 0)
grad_x = grad_xx[keepmask]
grad_y = grad_yy[keepmask]
norm = np.sqrt(grad_x* grad_x + grad_y * grad_y)
xx, yy = np.meshgrid(np.arange(map_8ds.occupancy().shape[0]),
                     np.arange(map_8ds.occupancy().shape[1]), indexing="ij")
x_ = xx[keepmask]
y_ = yy[keepmask]
plt.figure()
#plt.figure(figsize=(80,40))
gridshow(map_8ds.occupancy(), cmap=plt.cm.Greys)
plt.quiver(x_, y_, -grad_x/norm, -grad_y/norm, np.minimum(norm, 1), units='x', headaxislength=3, headwidth=3, headlength=3, width=0.1)
plt.colorbar()
#plt.savefig("costmap_gradient.png")

In [ ]:
norm_grid = np.sqrt(grad_xx* grad_xx + grad_yy * grad_yy)
ngrad_xx = -grad_xx / norm_grid
ngrad_xx[norm_grid == 0] = 0
ngrad_xx[ngrad_xx > 1] = 0
ngrad_yy = -grad_yy / norm_grid
ngrad_yy[norm_grid == 0] = 0
ngrad_yy[ngrad_yy > 1] = 0
plt.figure()
gridshow(ngrad_xx)
plt.figure()
gridshow(ngrad_yy)

In [ ]:
def compute_path(costmap, first):
    start = timer()
    r = np.roll(costmap, -1, axis=0) - costmap
    l = np.roll(costmap,  1, axis=0) - costmap
    u = np.roll(costmap, -1, axis=1) - costmap
    d = np.roll(costmap,  1, axis=1) - costmap
    edge_costs = np.stack([r, l, u, d], axis=-1)
    offsets = np.array([
                        [ 1, 0],
                        [-1, 0],
                        [0,  1],
                        [0, -1]])
    # trace path
    path = []
    jump_log = []
    
    path.append(first)

    while True:
        current = path[-1]
        current_idx = tuple(current.astype(int))
        choices = edge_costs[current_idx]
        cheapest = np.argsort(choices)
        best_cost = choices[cheapest[0]]
        second_best_cost = choices[cheapest[1]]
        selected_offset = offsets[cheapest[0]]
        has_jumped = False
        if best_cost >= 0:
            print("local minima")
            jump_log.append(has_jumped)
            break
        if second_best_cost < 0:
            # probabilistic jump
            rand = np.random.random()
            jump_chance = (second_best_cost 
                                     / (best_cost + second_best_cost))
            if rand <= jump_chance:
                selected_offset = offsets[cheapest[1]]
                has_jumped = True
        next_ = current + selected_offset
        path.append(next_)
        jump_log.append(has_jumped)

    print(timer() - start)
    return path, jump_log
path, jump_log = compute_path(dijkstra, first = np.array([70, 50]))
global_path = np.array(path)

In [ ]:
#plt.figure(figsize=(30,20))
plt.figure()
gridshow(costmap, vmax=max_dijkstra)
#gridshow(map_8ds.occupancy(), cmap=plt.cm.Greys)
#plt.quiver(x_, y_, -grad_x/norm, grad_y/norm, np.minimum(norm, 1), units='x', headaxislength=3, headwidth=3, headlength=3, width=0.1)
plt.scatter(np.array(path)[:,0], np.array(path)[:,1], c=np.array(jump_log) + 1,
           edgecolor="none", marker='s', cmap=plt.cm.autumn, s=10) #np.arange(len(path)))
plt.tight_layout()
#plt.savefig("plannedpath.png")

In [ ]:
plt.figure()
gridshow(map_.occupancy(), cmap=plt.cm.Greys)
plt.tight_layout()

In [ ]:
plt.figure()
gridshow(map_8ds.occupancy(), cmap=plt.cm.Greys)
plt.tight_layout()

## Debug Lidar

In [ ]:
virtual_pepper_debug = Virtual2DPepper(map_, debug=True)
virtual_pepper = Virtual2DPepper(map_)

In [ ]:
debug_out = []
virtual_pepper_debug.raytrace(virtual_pepper_debug.kTFBaseLinkToLidarRear, [], debug_out)
angles, ray_r, ray_i, ray_j, ray_x, ray_y, hits, ranges = debug_out

In [ ]:
start = timer()
ranges_no_debug = virtual_pepper.raytrace(virtual_pepper.kTFBaseLinkToLidarRear, [])
print("Raytrace took: {} [s]".format(timer() - start))

In [ ]:
plt.figure(figsize=(10,5))
gridshow(map_.occupancy(), cmap=plt.cm.Greys)
#plt.scatter(ranges_i, ranges_j)
plt.scatter(ray_i[::50,:].T, ray_j[::50,:].T, c=hits[::50,:].T, cmap=plt.cm.Pastel1,s=1,edgecolors="none")
ranges_x = ranges * np.cos(angles) + ray_x[0,0]
ranges_y = ranges * np.sin(angles) + ray_y[0,0]
plt.scatter(*map_.xy_to_ij(ranges_x, ranges_y), c='red', s=1,edgecolors="none")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.plot(ranges)
plt.plot(ranges_no_debug)

## Debug Map2D

In [ ]:
tic = timer()
map_.as_occupied_points_ij(numba=True)
toc = timer()
print("Occupied points (numba): {}s".format(abs(toc-tic)))

tic = timer()
map_.as_occupied_points_ij(numba=False)
toc = timer()
print("Occupied points: {}s".format(abs(toc-tic)))

## Debug Branch and Bound

In [ ]:
from branch_and_bound import BranchAndBound

In [ ]:
map_rot8ds = Map2D("/home/daniel/Desktop/", "gmapping_map2").as_coarse_map2d().as_coarse_map2d().as_coarse_map2d()

In [ ]:
bnb = BranchAndBound(map_8ds, rot_downsampling=2.)

In [ ]:
if False:
    import cProfile
    cProfile.run("score, pose, theta = bnb.branch_and_bound(map_rot8ds, match_threshold='1/2 matched points')")
else:
    score, pose, theta = 0, np.array([0,0]), 0

In [ ]:
print(score, pose, theta)

In [ ]:
plt.figure()
gridshow(map_rot8ds.occupancy(), cmap=plt.cm.Greys)
plt.figure()
hits = map_rot8ds.as_occupied_points_ij()
hits = bnb.rotate_points_around_map_center(hits, theta, map_rot8ds)
hits += pose
hits = hits[map_8ds.is_inside_ij(hits)]
gridshow(map_8ds.occupancy(), cmap=plt.cm.Greys)
is_confirmed_hit = map_8ds.occupancy()[tuple(hits[map_8ds.is_inside_ij(hits)].T)]
plt.scatter(hits[:,0], hits[:,1], c=is_confirmed_hit, s=1, cmap=plt.cm.bwr)


## Debug full

In [ ]:
#virtual_pepper = Virtual2DPepper(map_)

In [ ]:
#virtual_pepper.run()

## Localization

In [ ]:
local.occupancy_ = np.pad(local.occupancy(), ((0,0), (0,100)), 'constant', constant_values=0.5) # shape must be divisible by 8

In [ ]:
score, pose, theta = bnb.branch_and_bound(local.as_coarse_map2d().as_coarse_map2d().as_coarse_map2d(), 
                                          match_threshold='1/2 matched points',
                                         )


In [ ]:
plt.figure()
hits = local.as_coarse_map2d().as_coarse_map2d().as_coarse_map2d().as_occupied_points_ij()
hits = bnb.rotate_points_around_map_center(hits, theta, local.as_coarse_map2d().as_coarse_map2d().as_coarse_map2d())
hits += pose
gridshow(map_8ds.occupancy(), cmap=plt.cm.Greys)
plt.scatter(hits[:,0], hits[:,1], c=map_8ds.occupancy()[tuple(hits.T)], s=1, cmap=plt.cm.bwr)

In [ ]:
ij = bnb.rotate_points_around_map_center(local.as_coarse_map2d().as_coarse_map2d().as_coarse_map2d().xy_to_ij([[0,0]]), theta, local.as_coarse_map2d().as_coarse_map2d().as_coarse_map2d()) + pose

In [ ]:
pose2d_map8ds_local = np.block([ map_8ds.ij_to_xy(ij)[0], theta])

In [ ]:
from motion_planner import MotionPlanner
mp = MotionPlanner(map_)

## Local Path Planning

In [ ]:
global_path, global_jump_log = compute_path(dijkstra, ij[0])
global_path = np.array(global_path)
global_path_in_local_ij= local.xy_to_ij(pose2d.apply_tf(map_8ds.ij_to_xy(global_path), pose2d.inverse_pose2d(pose2d_map8ds_local)), clip_if_outside=False)

In [ ]:
plt.figure()
tic = timer()
local_tsdf = local.as_tsdf(0.5)
print("TSDF: {}".format(timer() - tic))
gridshow(local.as_tsdf(0.5))
plt.scatter(latest_hits[:,0], latest_hits[:,1], color='red', s=1)

In [ ]:
plt.figure()
gridshow(local.occupancy())
plt.scatter(global_path_in_local_ij[:,0], global_path_in_local_ij[:,1], c=np.array(global_jump_log) + 1,
           edgecolor="none", marker='s', cmap=plt.cm.autumn, s=10)
in_mask = local.is_inside_ij(global_path_in_local_ij)
valid_mask = local_tsdf[tuple(global_path_in_local_ij[in_mask].T)] > 0
local_goal = global_path_in_local_ij[in_mask][valid_mask][-1]

In [ ]:
plt.figure()
tic = timer()
local_dijkstra = local.dijkstra(local_goal, mask=local_tsdf < 0,
                           extra_costs = kDijkstraTSDFPenalty*(kPepperComfortZoneRadius - local_tsdf) * (local_tsdf < kPepperComfortZoneRadius))
print("Dijkstra: {}".format(timer() - tic))
gridshow(local_dijkstra, vmax=local_dijkstra[local_dijkstra < local_dijkstra.max()].max())
plt.scatter(local_goal[0], local_goal[1], edgecolor="none", marker='s', color='r', s=10)

## Path Smoothing

In [ ]:
path, jump_log = compute_path(local_dijkstra, first = np.array([150, 150]))
path_downsample_factor = int(len(path) / 30)
path_downsample_factor = 1 if path_downsample_factor == 0 else path_downsample_factor
path = path[::path_downsample_factor]
#path = global_path_in_local_ij[in_mask][valid_mask]
path = np.array(path).astype(float)

orig_path = np.array(path).copy()

min_tsdf_val = local_tsdf[tuple(path.T.astype(int))]
start_angle = 1.5
end_angle = np.arctan2(*(path[-1] - path[-2])[::-1])
local_comfort_tsdf = np.clip(local_tsdf, None, kPepperComfortZoneRadius)
    
from path_smoothing import path_smoothing, curvature_
tic = timer()
new_path, immobile = path_smoothing(path, local_comfort_tsdf, start_angle, end_angle)

print("Path smoothing: {}s".format(timer() - tic))
curvature = curvature_(local.ij_to_xy(new_path), start_angle, end_angle)

In [ ]:
plt.figure()
gridshow(local_tsdf)
plt.scatter(orig_path[:,0], orig_path[:,1], color='k', edgecolor="none", marker='s', s=10)
plt.scatter(new_path[:,0], new_path[:,1], color='r', edgecolor="none", marker='s', s=10)
plt.scatter(new_path[immobile][:,0], new_path[immobile][:,1], color='w', edgecolor="none", marker='s', s=10)

plt.figure()
plt.step(np.arange(len(curvature)), curvature, where='mid')
curvature[:5]

In [ ]:
kPepperMaxAccXY = 0.55
kPepperMaxVelXY = 0.55
vmax = np.sqrt(kPepperMaxAccXY/ np.abs(curvature))
plt.figure()
plt.step(np.arange(len(vmax)), vmax, where='mid')
plt.axhline(kPepperMaxVelXY, color='r')
plt.ylim([0,10])

In [ ]:
np.min(vmax)